In [2]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#loading the data
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1xfzvIbH779TooyrEo0ZH16LeI9QRIQHw'})

download.GetContentFile('train_LbELtWY.zip')
!unzip train_LbELtWY.zip

Archive:  train_LbELtWY.zip
   creating: train_LbELtWY/
  inflating: train_LbELtWY/train.csv  
   creating: train_LbELtWY/train/
  inflating: train_LbELtWY/train/1.png  
  inflating: train_LbELtWY/train/2.png  
  inflating: train_LbELtWY/train/3.png  
  inflating: train_LbELtWY/train/4.png  
  inflating: train_LbELtWY/train/5.png  
  inflating: train_LbELtWY/train/6.png  
  inflating: train_LbELtWY/train/7.png  
  inflating: train_LbELtWY/train/8.png  
  inflating: train_LbELtWY/train/9.png  
  inflating: train_LbELtWY/train/10.png  
  inflating: train_LbELtWY/train/11.png  
  inflating: train_LbELtWY/train/12.png  
  inflating: train_LbELtWY/train/13.png  
  inflating: train_LbELtWY/train/14.png  
  inflating: train_LbELtWY/train/15.png  
  inflating: train_LbELtWY/train/16.png  
  inflating: train_LbELtWY/train/17.png  
  inflating: train_LbELtWY/train/18.png  
  inflating: train_LbELtWY/train/19.png  
  inflating: train_LbELtWY/train/20.png  
  inflating: train_LbELtWY/train/21.png 

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [7]:
# the csv files contain the image file names along with the label
train = pd.read_csv('train_LbELtWY/train.csv')

# we will read all the training images and load them in a list and convert it to
# a numpy array
train_image = []

for i in tqdm(range(train.shape[0])):
  img = image.load_img('train_LbELtWY/train/'+train['id'][i].astype('str')+'.png', 
                         target_size=(48,48,1),
                         color_mode="grayscale")
  # convert image to an numpy array
  img =  image.img_to_array(img)
  # to keep the values from 0 to 1
  img = img/255
  train_image.append(img)

#load the data
X = np.array(train_image)
y = train['label'].values
y = to_categorical(y)

# train test split for validation

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, 
                                                    test_size=0.2)


100%|██████████| 981/981 [00:00<00:00, 2875.32it/s]


In [0]:
# define model structure
# We will create a simple architecture (simple lol) with 2 convolutional layers, one dense hidden layer and an output layer.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [13]:
#training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 784 samples, validate on 197 samples
Epoch 1/10
784/784 [==============================] - 8s 10ms/step - loss: 1.8025 - accuracy: 0.3508 - val_loss: 1.3339 - val_accuracy: 0.4619
Epoch 2/10
784/784 [==============================] - 7s 9ms/step - loss: 1.1181 - accuracy: 0.6097 - val_loss: 0.7457 - val_accuracy: 0.7107
Epoch 3/10
784/784 [==============================] - 7s 9ms/step - loss: 0.6892 - accuracy: 0.7666 - val_loss: 0.4909 - val_accuracy: 0.8629
Epoch 4/10
784/784 [==============================] - 7s 9ms/step - loss: 0.4635 - accuracy: 0.8559 - val_loss: 0.2503 - val_accuracy: 0.9391
Epoch 5/10
784/784 [==============================] - 7s 9ms/step - loss: 0.3099 - accuracy: 0.9018 - val_loss: 0.1809 - val_accuracy: 0.9340
Epoch 6/10
784/784 [==============================] - 7s 9ms/step - loss: 0.2177 - accuracy: 0.9260 - val_loss: 0.1323 - val_accuracy: 0.9594
Epoch 7/10
784/784 [==============================] - 7s 9ms/step - loss: 0.1609 - accuracy: 0.9477 -

In [14]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# save to gdrive 
model_json_file = drive.CreateFile({'title':'model.json'})
model_json_file.SetContentFile('model.json')
model_json_file.Upload()


# save to gdrive 
model_file = drive.CreateFile({'title':'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()



Saved model to disk
